In [1]:
import torchvision.models
import torchvision.models._utils as _utils

mov3lg = torchvision.models.mobilenet_v3_large()

In [2]:
print(mov3lg)

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [3]:
# 64 128 256
# 64 128 256 512
# 16 24 40 80 112 160 960

# 40 80 160
body5 = _utils.IntermediateLayerGetter(mov3lg.features, {'6':0, '10':1, '15':2})

In [4]:
body5

IntermediateLayerGetter(
  (0): ConvBNActivation(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): Hardswish()
  )
  (1): InvertedResidual(
    (block): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): ConvBNActivation(
        (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Identity()
      )
    )
  )
  (2): InvertedResidual(
    (block): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, 

In [5]:
import numpy as np
import cv2
import torch


In [6]:
from data import WiderFaceDetection, detection_collate, preproc, cfg_mnet, cfg_re50

img_dim = 840
rgb_mean = (104, 117, 123) # bgr order

dataset = WiderFaceDetection( './data/widerface/train/label.txt',preproc(img_dim, rgb_mean))

In [7]:
import torch.utils.data as data

batch_iterator = iter(data.DataLoader(dataset, 2, shuffle=True, num_workers=4, collate_fn=detection_collate))

In [8]:
images, targets = next(batch_iterator)

In [9]:
images.shape

torch.Size([2, 3, 840, 840])

In [10]:
targets

[tensor([[ 0.6174,  0.4787,  0.6617,  0.5268,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000, -1.0000],
         [ 0.6599,  0.4584,  0.7061,  0.5028,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000, -1.0000],
         [ 0.8503,  0.4566,  0.8909,  0.4991,  0.8634,  0.4711,  0.8792,  0.4722,
           0.8721,  0.4803,  0.8658,  0.4874,  0.8775,  0.4877,  1.0000],
         [ 0.8835,  0.4658,  0.9168,  0.5046,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000, -1.0000]]),
 tensor([[0.3858, 0.0549, 0.8742, 0.6752, 0.6035, 0.2397, 0.8014, 0.2707, 0.7160,
          0.4027, 0.5530, 0.4570, 0.7626, 0.4881, 1.0000]])]

In [11]:
body5outputs = body5(images)
body5outputs

OrderedDict([(0,
              tensor([[[[ 1.1721e+00,  6.9228e-02, -3.9747e-01,  ...,  8.7818e-01,
                         -1.1011e-01,  2.8548e-01],
                        [ 4.7327e-01, -1.0635e+00,  2.7001e-02,  ..., -8.0862e-01,
                         -7.9550e-01, -1.6185e-01],
                        [ 1.1909e+00, -2.7725e-01,  6.9662e-01,  ...,  5.5011e-01,
                          1.0003e+00,  6.8787e-01],
                        ...,
                        [-4.8151e-01,  5.1347e-02, -1.1363e+00,  ...,  1.2122e-03,
                         -4.8677e-01, -2.3148e-01],
                        [ 9.6476e-01,  6.3809e-01,  1.6118e+00,  ..., -4.2627e-02,
                         -9.9582e-02,  4.4157e-01],
                        [ 9.6395e-01,  1.5906e+00,  1.0402e+00,  ..., -7.9937e-01,
                         -8.5396e-01,  1.9993e-01]],
              
                       [[-8.7627e-01, -5.4330e-01, -2.5792e-02,  ..., -1.8612e-01,
                         -6.4415e-02, -1.1243

In [12]:
body5outputs.keys()

odict_keys([0, 1, 2])

In [13]:
body5outputs.values()

odict_values([tensor([[[[ 1.1721e+00,  6.9228e-02, -3.9747e-01,  ...,  8.7818e-01,
           -1.1011e-01,  2.8548e-01],
          [ 4.7327e-01, -1.0635e+00,  2.7001e-02,  ..., -8.0862e-01,
           -7.9550e-01, -1.6185e-01],
          [ 1.1909e+00, -2.7725e-01,  6.9662e-01,  ...,  5.5011e-01,
            1.0003e+00,  6.8787e-01],
          ...,
          [-4.8151e-01,  5.1347e-02, -1.1363e+00,  ...,  1.2122e-03,
           -4.8677e-01, -2.3148e-01],
          [ 9.6476e-01,  6.3809e-01,  1.6118e+00,  ..., -4.2627e-02,
           -9.9582e-02,  4.4157e-01],
          [ 9.6395e-01,  1.5906e+00,  1.0402e+00,  ..., -7.9937e-01,
           -8.5396e-01,  1.9993e-01]],

         [[-8.7627e-01, -5.4330e-01, -2.5792e-02,  ..., -1.8612e-01,
           -6.4415e-02, -1.1243e+00],
          [-2.5136e-01, -4.8535e-01,  5.4443e-01,  ...,  5.4259e-01,
            7.8116e-01,  3.4117e-01],
          [-1.4178e+00, -8.5801e-01,  1.0186e+00,  ..., -5.5463e-02,
            9.2877e-02,  1.1118e+00],
      

In [14]:
body5outputsvalues = list(body5outputs.values())

In [15]:
body5outputsvalues[0].shape

torch.Size([2, 40, 105, 105])

In [16]:
body5outputsvalues[1].shape

torch.Size([2, 80, 53, 53])

In [17]:
body5outputsvalues[2].shape

torch.Size([2, 160, 27, 27])

In [69]:
from models.retinaface import RetinaFace

cfg = cfg_mnet

mnet1 = RetinaFace(cfg)


In [70]:
import torch

with torch.no_grad():
    mnet1outputs = mnet1(images)

In [71]:
import numpy as np

len(mnet1outputs)

3

In [72]:
mnet1outputs[0].shape

torch.Size([2, 29126, 4])

In [73]:
mnet1outputs[1].shape

torch.Size([2, 29126, 2])

In [74]:
mnet1outputs[2].shape

torch.Size([2, 29126, 10])

In [19]:
import torch
import torch.nn as nn
import torchvision.models.detection.backbone_utils as backbone_utils
import torchvision.models._utils as _utils
import torch.nn.functional as F
from collections import OrderedDict

from models.net import MobileNetV1 as MobileNetV1
from models.net import FPN as FPN
from models.net import SSH as SSH

def conv_bn(inp, oup, stride = 1, leaky = 0):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.LeakyReLU(negative_slope=leaky, inplace=True)
    )

def conv_bn_no_relu(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
    )

def conv_bn1X1(inp, oup, stride, leaky=0):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, stride, padding=0, bias=False),
        nn.BatchNorm2d(oup),
        nn.LeakyReLU(negative_slope=leaky, inplace=True)
    )

def conv_dw(inp, oup, stride, leaky=0.1):
    return nn.Sequential(
        nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
        nn.BatchNorm2d(inp),
        nn.LeakyReLU(negative_slope= leaky,inplace=True),

        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.LeakyReLU(negative_slope= leaky,inplace=True),
    )

class SSH_mv3(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(SSH, self).__init__()
        assert out_channel % 4 == 0
        leaky = 0
        if (out_channel <= 64):
            leaky = 0.1
        self.conv3X3 = conv_bn_no_relu(in_channel, out_channel//2, stride=1)

        self.conv5X5_1 = conv_bn(in_channel, out_channel//4, stride=1, leaky = leaky)
        self.conv5X5_2 = conv_bn_no_relu(out_channel//4, out_channel//4, stride=1)

        self.conv7X7_2 = conv_bn(out_channel//4, out_channel//4, stride=1, leaky = leaky)
        self.conv7x7_3 = conv_bn_no_relu(out_channel//4, out_channel//4, stride=1)

    def forward(self, input):
        conv3X3 = self.conv3X3(input)

        conv5X5_1 = self.conv5X5_1(input)
        conv5X5 = self.conv5X5_2(conv5X5_1)

        conv7X7_2 = self.conv7X7_2(conv5X5_1)
        conv7X7 = self.conv7x7_3(conv7X7_2)

        out = torch.cat([conv3X3, conv5X5, conv7X7], dim=1)
        out = F.relu(out)
        return out

class ClassHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(ClassHead,self).__init__()
        self.num_anchors = num_anchors
        self.conv1x1 = nn.Conv2d(inchannels,self.num_anchors*2,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 2)

class BboxHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(BboxHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*4,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 4)

class LandmarkHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(LandmarkHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*10,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 10)


class RetinaFace_mobilenetv3lg(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(RetinaFace_mobilenetv3lg,self).__init__()
        self.phase = phase
        backbone = None
        backbone = torchvision.models.mobilenet_v3_large()
        # backbone.features[4].block[3][0].stride=(1,1)
        # backbone.features[9].block[3][0].stride=(1,1)
        print(backbone)

        self.body = _utils.IntermediateLayerGetter(backbone.features, {'6':0, '10':1, '15':2})
        # in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            40,
            80,
            160,
        ]
        # out_channels = cfg['out_channel']
        out_channels = 64
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=out_channels)
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=out_channels)
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=out_channels)

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead

    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]
        print("feature1.shape: ", feature1.shape)
        print("feature2.shape: ", feature2.shape)
        print("feature3.shape: ", feature3.shape)

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

In [20]:
mnetv3 = RetinaFace_mobilenetv3lg()

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [21]:
with torch.no_grad():
    mnetv3outputs = mnetv3(images)

feature1.shape:  torch.Size([2, 64, 105, 105])
feature2.shape:  torch.Size([2, 64, 53, 53])
feature3.shape:  torch.Size([2, 64, 27, 27])


In [22]:
len(mnetv3outputs)

3

In [23]:
mnetv3outputs[0].shape

torch.Size([2, 29126, 4])

In [24]:
mnetv3outputs[1].shape

torch.Size([2, 29126, 2])

In [25]:
mnetv3outputs[2].shape

torch.Size([2, 29126, 10])

In [31]:
cfgres = cfg_re50

res50fnet = RetinaFace(cfgres)

In [32]:
with torch.no_grad():
    res50fnetoutputs = res50fnet(images)

feature1.shape:  torch.Size([2, 256, 105, 105])
feature2.shape:  torch.Size([2, 256, 53, 53])
feature3.shape:  torch.Size([2, 256, 27, 27])


In [33]:
len(res50fnetoutputs)

3

In [34]:
res50fnetoutputs[0].shape

torch.Size([2, 29126, 4])

In [35]:
res50fnetoutputs[1].shape

torch.Size([2, 29126, 2])

In [36]:
res50fnetoutputs[2].shape

torch.Size([2, 29126, 10])

In [37]:
len(targets)

2

In [38]:
targets[0].shape

torch.Size([9, 15])

In [39]:
targets[1].shape

torch.Size([1, 15])